In [3]:
# Prep ... import libraries
import pandas as pd
import requests
from pathlib import Path

# Import API key & set base URL
from api_keys import walkscore_key
url="https://api.walkscore.com/score?format=json&address=skip"

# Define variables
FindState = "CA"

In [5]:
# Import CSV
data_df = pd.read_csv("../resources/obesity_data_ALL_states.csv")
data_df = pd.DataFrame(data_df)

# Insert new column at end to import Walk Score
data_df["Walk Score"]=''

# Clean up dataframe by removing any rows with no data
data_df_cleaned=data_df.dropna()

# data_df_cleaned["StateAbbr"].value_counts()

In [6]:
# Let's do API calls for just ONE state
# We define FindState in first block...
one_state = data_df_cleaned.loc[data_df_cleaned["StateAbbr"] == FindState, :]

# reset index because of dropped rows
one_state.reset_index(drop=True, inplace=True)

one_state.head(10)

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,CA,Los Angeles,0644000-06037190902,644000,6037190902,30.2,3960,34.094395,-118.319013,
1,CA,Stockton,0675000-06077003305,675000,6077003305,26.2,3704,38.001905,-121.307265,
2,CA,Corona,0616350-06065048200,616350,6065048200,24.7,4301,33.888935,-117.540737,
3,CA,Los Angeles,0644000-06037211122,644000,6037211122,24.8,3075,34.072679,-118.289223,
4,CA,San Francisco,0667000-06075010700,667000,6075010700,14.8,5592,37.798514,-122.410047,
5,CA,Los Angeles,0644000-06037199110,644000,6037199110,29.0,2710,34.079362,-118.197404,
6,CA,Chula Vista,0613392-06073012900,613392,6073012900,25.2,3374,32.627420,-117.067604,
7,CA,Long Beach,0643000-06037571800,643000,6037571800,20.4,3190,33.833850,-118.197208,
8,CA,Santa Ana,0669000-06059074701,669000,6059074701,28.8,8346,33.730917,-117.895512,
9,CA,Fremont,0626000-06001441927,626000,6001441927,13.0,4103,37.546502,-121.976577,


In [8]:
# Now let's iterate thru that ONE state and get the data

for index,row in one_state.iterrows():

    try:
        lat = str(row["Lat"])
        lon = str(row["Lon"])
        query_url = url+"&lat="+lat+"&lon="+lon+"&wsapikey="+walkscore_key
        walk_response = requests.get(query_url)
        walk_json = walk_response.json() 
        if walk_json["status"] == 41:
            print("Daily API quota exceeded")
            print("^^^^^^^^^^^^^^^^^^^^^^^^")
            break
            
        print("Processing LAT %s, LON, %s" % (lat, lon))
        one_state.loc[index, "Walk Score"] = walk_json["walkscore"]
    
    except:
        print("Data not found. Skipping")
        pass


Processing LAT 34.0943950467, LON, -118.31901303
Processing LAT 38.0019053412, LON, -121.307265093
Processing LAT 33.888934791, LON, -117.540737279
Processing LAT 34.0726793937, LON, -118.289223419
Processing LAT 37.7985139858, LON, -122.410046646
Processing LAT 34.0793616083, LON, -118.197403908
Processing LAT 32.6274203123, LON, -117.067603902
Processing LAT 33.8338498947, LON, -118.197207556
Processing LAT 33.7309169673, LON, -117.89551245
Processing LAT 37.5465021171, LON, -121.976577005
Processing LAT 34.0475521969, LON, -118.420198157
Processing LAT 37.9762527602, LON, -121.794017649
Processing LAT 38.7796448786, LON, -121.286281979
Processing LAT 34.0164074473, LON, -118.331152283
Processing LAT 33.8793825678, LON, -117.893411571
Processing LAT 34.0311671054, LON, -117.661876503
Processing LAT 33.6908641043, LON, -117.843954538
Processing LAT 36.767354277, LON, -119.872420655
Processing LAT 39.7235271863, LON, -121.781409059
Processing LAT 32.8124713464, LON, -116.926474748
Proc

In [10]:
one_state.head()

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,CA,Los Angeles,0644000-06037190902,644000,6037190902,30.2,3960,34.094395,-118.319013,86
1,CA,Stockton,0675000-06077003305,675000,6077003305,26.2,3704,38.001905,-121.307265,69
2,CA,Corona,0616350-06065048200,616350,6065048200,24.7,4301,33.888935,-117.540737,7
3,CA,Los Angeles,0644000-06037211122,644000,6037211122,24.8,3075,34.072679,-118.289223,91
4,CA,San Francisco,0667000-06075010700,667000,6075010700,14.8,5592,37.798514,-122.410047,100


In [28]:
# Save output to CSV
output_csv = f'../resources/{FindState}_complete.csv'
one_state.to_csv(output_csv, index=False)